In [1]:
from datetime import timedelta, datetime, tzinfo
import numpy as np
import matplotlib.pyplot as plt
import h5py
import argparse
import sharp
import sharpnsls2
# make graphics inline
%matplotlib inline
# %matplotlib notebook

# Input parameters

In [2]:
fname = '../data/scan_37466.h5' # HXN input file
ccd_pixel_um = 55 # detector pixel size (um)

niters = 1 # number of iterations
start_ave = 0.8  ## average starting iteration

alpha = 1.e-8
beta  = 0.9 # reconstruction feedback parameters

amp_max = 1.00  # up limit of allowed object amplitude range
amp_min = 0.95  # low limit of allowed object amplitude range
pha_max = 0.01  # up limit of allowed object phase range
pha_min = -0.1  # low limit of allowed object amplitude range

# Reading a HXN input file

In [3]:
print("reading a HXN input file ...");
t1 = datetime.now();

f = h5py.File(fname, "r")

diffamp = np.array(f['diffamp'])   # frames      

# scan data
scan_points = np.array(f['points'])     
x_range = np.asscalar(np.array(f['x_range']))
y_range = np.asscalar(np.array(f['y_range']))
dr_x = np.asscalar(np.array(f['dr_x']))
dr_y = np.asscalar(np.array(f['dr_y']))

z_m = np.asscalar(np.array(f['z_m'])) # detector-to-sample distance (m)
lambda_nm = np.asscalar(np.array(f['lambda_nm'])) # wavelength (nm)

f.close()

t2 = datetime.now();
print ("processing time: ", (t2 - t1))

nz, nx, ny = np.shape(diffamp)
print(nx,ny,nz, scan_points.shape, z_m, lambda_nm, x_range, y_range, dr_x, dr_y)

reading a HXN input file ...
processing time:  0:00:03.925877
128 128 40000 (2, 40000) 0.5 0.10333333333333333 1.99 1.99 0.01 0.01


# Select a subset with  20 K frames

In [4]:
# frames
diffamp = diffamp[0::2]
nz, nx, ny = np.shape(diffamp)

# scan
points  = scan_points[0:2, 0::2]

print(nx,ny,nz, points.shape)

128 128 20000 (2, 20000)


# Set the input parameters

In [5]:
sharpNSLS2 = sharpnsls2.PySharpNSLS2()

sharpNSLS2.setZ(z_m)
sharpNSLS2.setLambda(lambda_nm)
sharpNSLS2.setPixelSize(ccd_pixel_um)

# Load frames

In [6]:
frames = []
emptyFrames = []

for i in range(0, nz):
    amp2 = diffamp[i]**2
    if np.sum(amp2) > 0.:
        frames.append(np.fft.fftshift(amp2))
    else :
        emptyFrames.append(i)

# plt.imshow(frames[0])
print(frames[0].shape, frames[0][0].dtype, len(emptyFrames))

sharpNSLS2.setFrames(np.array(frames).astype(np.float32))

(128, 128) float64 0


# Load an initial probe

In [7]:
# generate from frames
s = diffamp[0,:,:]
init_prb = np.fft.fftshift(np.fft.fftn(s)) / np.sqrt(np.size(s))

# check with recon
recon_init_prb = np.load("../recon/init_prb.npy")
prb_diff = init_prb - recon_init_prb

print(init_prb.shape, init_prb[0].dtype, np.max(init_prb), np.max(prb_diff))

# load
sharpNSLS2.setInitProbe(np.asarray(init_prb).astype(np.complex64))

(128, 128) complex128 (2376.67739367+0j) 0j


# Load the scan points

In [8]:
# scan direction and geometry correction handling
# recon-oriented operations

det_side = nx
real_pixel_size_um = 1.e6*z_m*lambda_nm*1e-3/(det_side*ccd_pixel_um)

print('real pixel size [um]:', real_pixel_size_um)

recon_points = -1*points
recon_points[0,:] *=-1 * np.abs(np.cos(15.*np.pi/180.))

xs = np.round(recon_points[0]/real_pixel_size_um) 
ys = np.round(recon_points[1]/real_pixel_size_um) 

xs -= min(xs)
ys -= min(ys)

xmax = max(xs)
ymax = max(ys)

xs = xs*real_pixel_size_um 
ys = ys*real_pixel_size_um 

real_translation = np.column_stack((ys, xs, np.zeros(ys.size)))
real_translation = real_translation * 1.0e-6 # recon_points in um

sharpNSLS2.setScan(real_translation.astype(np.float64))

real pixel size [um]: 0.007339015151515151


# Load an initial object

In [9]:
nx_obj = int(det_side + xmax)
ny_obj = int(det_side + ymax)

init_obj = np.random.uniform(0, 1.0, (nx_obj, ny_obj)) * np.exp(np.random.uniform(0, np.pi, (nx_obj, ny_obj))*1.j)

print('sharp-nsls2 obj:', nx_obj, ny_obj, init_obj.shape)

# recon input object

init_obj = np.load("../recon/init_obj.npy")

recon_nx_obj = nx_obj + 30
recon_ny_obj = ny_obj + 30

recon_nx_obj = np.int(recon_nx_obj + np.mod(recon_nx_obj, 2))
recon_ny_obj = np.int(recon_ny_obj + np.mod(recon_ny_obj, 2))

print('recon obj:', recon_nx_obj, recon_ny_obj, init_obj.shape)

init_obj = init_obj[15:nx_obj+15, 15:ny_obj+15]

sharpNSLS2.setInitObject(init_obj.astype(np.complex64))

sharp-nsls2 obj: 389 399 (389, 399)
recon obj: 420 430 (420, 430)


# Set the SHARP-NSLS2 parameters

In [10]:
sharpNSLS2.setStartUpdateObject(0)
sharpNSLS2.setStartUpdateProbe(2)

sharpNSLS2.setAmpMax(amp_max)
sharpNSLS2.setAmpMin(amp_min)
sharpNSLS2.setPhaMax(pha_max)
sharpNSLS2.setPhaMin(pha_min)

sharpNSLS2.setAlpha(alpha)
sharpNSLS2.setBeta(beta)

sharpNSLS2.setChunks(1)

# Initialize the SHARP-NSLS2 containers

In [11]:
sharpNSLS2.init()

0